# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
display(data.head())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [7]:
data.duplicated().sum()

153

In [8]:
data['Возраст'] = data['Возраст'].astype(int)
data['Зарплата'] = data['Зарплата'].astype(int)
data.dtypes

Пол                  int64
Возраст              int64
Зарплата             int64
Члены семьи          int64
Страховые выплаты    int64
dtype: object

По итогам предобработки данных данные приведены к единому типу, пропусков в данных не выявлено, количество дубликатов незначительно, поэтому не требует углубленной проверки.

In [9]:
corr = data.corr()
corr.style.background_gradient(cmap='Greys')

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Ответьте на вопрос и обоснуйте решение.

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

a. Изменится. Приведите примеры матриц.

b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Минимальное значение MSE получается, когда веса равны этой величине:

$$ w = (X^T)^{-1}X^Ty$$

Допустим, матрица $X(1)$ равна произведению матрицы $X$ на обратимую матрицу $P$

$$X(1) = X * P$$

Тогда:

$$ w(1) = ((XP)^T(XP))^{-1}(XP)^Ty$$

Далее раскроем первое произведение $(XP)^T$:

$$w(1) = (P^T X^TXP)^{-1}P^TX^Ty$$

Перегруппируем множители в скобках, чтобы получить произведение трех множителей:

$$w(1) = (P^T(X^TX)^{-1}P)^{-1}P^TX^Ty$$

Раскроем скобки $(P^T(X^TX)^{-1}P)$:

$$w(1) = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty$$

В соответствии с условием матрица $P$ - обратимая, соответственно, произведение $(P^T)^{-1}(P^T)$ является единичной матрицей $E$, следовательно получаем:

$$w(1) = P^{-1}(X^TX)^{-1}EX^{-1}y = P^{-1}(X^TX)^{-1}X^Ty$$

Таким образом в правой части уравнения мы имеем формулу $w$:

$$w(1) = P^{-1}(X^TX)^{-1}X^Ty = P^{-1}w$$

Предсказания модели линейной регрессии рассчитываются по формуле:

$$ a = Xw $$

Подставим в эту формулу значения $X(1)$ и $w(1)$ для того, чтобы на их основе вычислить предсказания модели $a(1)$:

$$a(1) = X(1)w(1) = PP^{-1}w$$

В соответствии с условием матрица $P$ - обратимая, следовательно $PP^{-1} = E$, получаем:

$$a = XPP^{-1}w = XEw = Xw = a $$

Таким образом, мы доказали, что предсказания $a1$ для матрицы умноженной на обратимую матрицу $P$ равны предсказаниям $a$.
Соответственно, мы можем утверждать, что параметры линейной регрессии в исходной задаче $w$ и в преобразованной $w1$ связаны следующим образом:

$$w1 = P^{-1}w$$

**Ответ:** b. Качество линейной регрессии не изменится. 

**Вывод**

При умножении признаков на обратимую матрицу предсказания модели линейной регрессии не изменятся, однако изменятся веса признаков. Таким образом необходимо переобучить модель в соответствии с измененными весами.



## Алгоритм преобразования

**Алгоритм**

Сравним метрики R2 на исходных и зашифрованных признаках.

В целях защиты данных и и сохранения качества обучения модели линейной регрессии, нужно провести процедуру умножения признаков на обратимую матрицу $P$. Следовательно, алгоритм преобразования данных должен включать в себя создание обратимой матрицы $P$ и ее умножения на матрицу признаков.

В соответствии с условием, исходом умножения обратимой матрицы на матрицу признаков должна быть матрица, размер которой равен размеру исходной матрицы. Следовательно, обратимая матрица $P$ должна быть квадратной, равной по размеру количеству признаков. Соответственно в конкретном случае матрица должна иметь размер 4х4.

Сгенерируем случайную матрицу:

In [10]:
p_matrix = np.random.randint(100,size = (4, 4))
p_matrix

array([[52, 21, 56, 80],
       [30, 30, 87, 32],
       [95,  8, 73,  4],
       [34, 59, 64, 95]])

Проверим обратимость матрицы:

In [11]:
p_inverted = np.linalg.inv(p_matrix)
p_inverted

array([[-0.00266255, -0.015387  ,  0.01440238,  0.00681872],
       [-0.03656091, -0.0109693 ,  0.01130534,  0.03400704],
       [ 0.00641194,  0.02133897, -0.00583123, -0.01234186],
       [ 0.01933954, -0.00205629, -0.00824734, -0.00471971]])

Известно, что предсказания модели после преобразования не изменятся, следовательно, качество модели не изменится, поскольку качество модели - это производная от предсказаний и целевого признака.

В качестве иллюстрации приведем пример, для которого произведем расчет вектора предсказаний $a$ и $a1$ по формулам, а так же проведем их сравнение друг с другом.

Выделим обучающие и целевой признаки и проведем масштабирование признаков:

In [12]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

Произведем расчет $w$ для исходных признаков по формуле $w = (X^T X)^{-1} X^T y$

In [13]:
w = np.linalg.inv(features.T.dot(features)).dot(features.T).dot(target)
display(w)

array([-4.43854736e-02,  2.33356252e-02, -1.17739062e-05, -4.55168115e-02])

Расчитаем вектор предсказаний по исходным признакам по формуле $𝑎=𝑋𝑤$

In [14]:
a = features @ w

Преобразуем признаки:

In [15]:
features_p = features @ p_matrix

Расчитаем $𝑤1$  для преобразованных признаков по формуле  $𝑤=(𝑋^𝑇𝑋)^{−1}𝑋^𝑇𝑦$:

In [16]:
w1 = np.linalg.inv(features_p.T.dot(features_p)).dot(features_p.T).dot(target)

Расчитаем вектор предсказаний по преобразованным признакам по формуле $𝑎=𝑋𝑤$:

In [17]:
a1 = features_p @ w1

Вычислим вектор разниц между предсказаниями по исходным признакам $𝑎$ и преобразованным $a1$:
Просуммируем все элементы полученного вектора для проверки расхождений:

In [18]:
v = a - a1

In [19]:
print(v.sum())

-6.250952944919378e-05


Полученный результат говорит о том, что расхождение минимально. Из чего мы можем сделать вывод, что $a=a1$

## Проверка алгоритма

Обучим модель линейной регрессии для исходных данных и оценим ее метрику R2_score

In [20]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
mse = mean_squared_error(target, predictions)
print(mse)
print(r2_score(target, predictions))

0.12334688937098945
0.42494550308169177


Преобразуем признаки умножив на матрицу P и рассчитаем R2_Score

In [21]:
features_pre = features @ p_matrix

In [22]:
model = LinearRegression()
model.fit(features_pre, target)
predictions = model.predict(features_pre)
mse = mean_squared_error(target, predictions)
print(mse)
print(r2_score(target, predictions))

0.12334688937098913
0.4249455030816932


Метрики R2_score для двух моделей имеют незначительные различия, из чего мы можем сделать вывод, что умножение матрицы признаков на обратимую матрицу позволяет зашифровать персональные данные пользователей не ухудшая качество предсказаний модели линейной регрессии.